<a href="https://colab.research.google.com/github/HarshithaShetty27/Sentiment-analysis/blob/main/Sentiment_analysis_IMDB_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

***IMPORTING THE DEPENDENCIES***

In [ ]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import  train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

***DATA COLLECTION - KAGGLE***

In [ ]:
kaggle_dictionary = json.load(open('kaggle.json'))

In [ ]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [ ]:
#kaggle credentials as env
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other


In [ ]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [ ]:
#unzip
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

In [ ]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


***LOAD THE DATASET***

In [ ]:
data = pd.read_csv("/content/IMDB Dataset.csv")

In [ ]:
data.shape # rows x columns

(50000, 2)

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [ ]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
data.replace({"sentiment" : {"positive" : 1, "negative": 0}}, inplace=True)

<ipython-input-16-c94ce39755b8>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment" : {"positive" : 1, "negative": 0}}, inplace=True)


In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
data["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [ ]:
#split
train_data , test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


***DATA PREPROCESSING***

In [ ]:
#Tokenization
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [ ]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [ ]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [ ]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [ ]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


LSTM - long short term memory

In [ ]:
#build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

***TRAINING THE MODEL***

In [ ]:
model.fit(X_train, Y_train, batch_size=64, epochs=10, validation_split=0.2)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 330s 650ms/step - accuracy: 0.7430 - loss: 0.5123 - val_accuracy: 0.8123 - val_loss: 0.4191
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 380s 647ms/step - accuracy: 0.8450 - loss: 0.3666 - val_accuracy: 0.8401 - val_loss: 0.3712
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 324s 648ms/step - accuracy: 0.8773 - loss: 0.3035 - val_accuracy: 0.7861 - val_loss: 0.4598
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 384s 653ms/step - accuracy: 0.8805 - loss: 0.2980 - val_accuracy: 0.8620 - val_loss: 0.3446
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 379s 648ms/step - accuracy: 0.8795 - loss: 0.2895 - val_accuracy: 0.8740 - val_loss: 0.3202
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 388s 661ms/step - accuracy: 0.9198 - loss: 0.2082 - val_accuracy: 0.8759 - val_loss: 0.3236
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 320s 640ms/step - accuracy: 0.9320 - loss: 0.1826 - val_accuracy: 0.8720 - val_loss: 0.3389
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 326s 651ms/step - accuracy: 0.9385 -

***MODEL EVALUATION***

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Loss : {loss}")
print(f"Accuracy : {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 86ms/step - accuracy: 0.8680 - loss: 0.4028
Loss : 0.3983001112937927
Accuracy : 0.8751999735832214


***PREDICTIVE SYSTEM***

In [ ]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [ ]:
new_review = "This movie was amazing!"
sentiment = predict_sentiment(new_review)
print(f"Predicted Sentiment: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Predicted Sentiment: positive


In [ ]:
new_review = "Hated this movie"
sentiment = predict_sentiment(new_review)
print(f"Predicted Sentiment: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Predicted Sentiment: negative


In [ ]:
new_review = "This movie was very boring"
sentiment = predict_sentiment(new_review)
print(f"Predicted Sentiment: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Predicted Sentiment: negative


In [ ]:
new_review = "Great visuals"
sentiment = predict_sentiment(new_review)
print(f"Predicted Sentiment: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Predicted Sentiment: positive


In [ ]:
new_review = "This movie was not good"
sentiment = predict_sentiment(new_review)
print(f"Predicted Sentiment: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
Predicted Sentiment: negative
